In [ ]:
import os 
import imageio
from IPython.display import display, Image
from sklearn.preprocessing import OneHotEncoder
import numpy as np 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
#for using in google colab
!unzip "drive/MyDrive/trainimages.zip" -d "/content/data/"

In [ ]:
train_path = 'data/train/images/'

#ignore .ipynb_checkpoints
classes = [f for f in os.listdir(train_path) if not f.startswith('.')]

train_x, train_y = [], []
#128x87 with 1 channel
image_shape = (128,87,1)

for c in classes:
  label = classes.index(c)
  print(label)
  class_path = train_path + c + '/'
  images = os.listdir(train_path+c+'/')
  for im in images:
    #display(Image(filename=class_path + im))
    x = np.array(imageio.imread(class_path+im))
    x = np.expand_dims(x,axis=0)
    x = np.resize(x,image_shape)
    train_x.append(x)
    train_y.append(label)

In [ ]:
train_x = np.array(train_x)
train_y = np.array(train_y).reshape(-1,1)

onehot = OneHotEncoder()
train_y = onehot.fit_transform(train_y).toarray()
print(train_x.shape)
print(train_y.shape)
num_classes = train_y.shape[1]

In [ ]:
shuffler = np.random.permutation(len(train_x))

train_x = train_x[shuffler]

train_y = train_y[shuffler]

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=image_shape),
        layers.Conv2D(128,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.Conv2D(128,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.Conv2D(128,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        #layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(256,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.Conv2D(256,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.Conv2D(256,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(512,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.Conv2D(512,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.Conv2D(512,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(512,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),    
        layers.Conv2D(512,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),   
        layers.Conv2D(512,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),     
        layers.Flatten(),
        layers.Dense(1024,activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.Dense(512,activation="relu",kernel_regularizer=regularizers.l2(4e-5)),

        layers.Dense(num_classes,activation="softmax",kernel_regularizer=regularizers.l2(4e-5))
    ],
    name="cnn-12"
)

model.summary()

In [ ]:
#opt = keras.optimizers.SGD(learning_rate = 0.01, momentum=0.9,nesterov=True)

opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,loss="categorical_crossentropy",metrics=['accuracy'])

In [ ]:
#model = keras.models.load_model('weights.h5')
early_stop = EarlyStopping(monitor='val_loss',min_delta=.00001, patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5',monitor='val_accuracy', save_best_only=True,mode='max')
#model.fit(train_x,train_y,batch_size=50,epochs=100, callbacks=[early_stop,checkpoint], validation_split = 0.15)
model.fit(train_x,train_y,batch_size=50,epochs=100, validation_split = 0.1)

In [ ]:
model.save('weights.h5')